# Imports

In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
import random
from scipy import stats
import plotly.express as px
from sklearn.metrics import mean_squared_error, r2_score

import os

In [3]:
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model_canada/model/')

In [4]:
from pipeline_config import *
from hgru_data_preprocess.preprocess_config import *

In [5]:
torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

# Read Results Data

In [6]:
with open(test_predictions_path, 'rb') as f:
    prediction_dic = pickle.load(f)

with open(canada_path, 'rb') as f:
    raw_dataset_dict = pickle.load(f)
    
with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

In [7]:
def create_test_dataframe(raw_dataset_dict: dict):
    test_dict = {}
    for key in raw_dataset_dict.keys():
        df = raw_dataset_dict[key][['Category', 'Date', 'Year', 'Indent', 'Inflation t+1']]
        df.dropna(inplace=True)
        df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
        target_df = df[df['Year'] > Year]
        test_dict[key] = target_df
    return test_dict

test_dict = create_test_dataframe(raw_dataset_dict)


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_5650/1314998454.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_5650/1314998454.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_5650/1314998454.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [8]:
def get_df_with_predictions(prediction_dic: dict, dict_of_categories_df:dict) -> dict:
    all_data_dict = {}
    for key in list(prediction_dic.keys()):
        predictions = prediction_dic[key]
        prediction_df =  pd.DataFrame(predictions.flatten().detach().numpy())
        prediction_df.rename(columns = {0: 'Prediction'}, inplace=True)
        dict_of_categories_df[key] = dict_of_categories_df[key].reset_index(drop=True)
        all_data_dict[key] = pd.concat([dict_of_categories_df[key], prediction_df], axis=1)
    return all_data_dict

all_data_test_dict=get_df_with_predictions(prediction_dic, test_dict)

In [9]:
#categories_per_indent_dict ={key: categories_per_indent_dict[key] for key in categories_per_indent_dict.keys() if key in [0,1,2,3]}
#cat_ids = list(categories_per_indent_dict.values())

#flat_cat_id = [item for sublist in cat_ids for item in sublist]

#all_categories = []
#for cat_id in flat_cat_id:
#    all_categories.append(category_id_to_name_dict[cat_id])

In [10]:
def plot_results(all_data_dict, categories):
    category_samples = random.sample(categories, 20)+['All-items']
    #category_samples = categories
    for category in category_samples:
        category_df = all_data_dict[category]
        y_pred = category_df['Prediction'].values
        y_actual = category_df['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)
        print(f'Category is: {category}')
        print(f'RMSE is: {np.sqrt(mse)}')

        fig = px.line(category_df, x="Date", y=["Actual", "Prediction"], title=f'{category} - Actual VS Prediction')
        fig.show()

#plot_results(all_data_test_dict, all_categories)

In [24]:
def avg_rmse(all_data_test_dict):
    mse_lst = []
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)
        mse_lst.append(mse)
    
    rmse_list = list(map(np.sqrt,mse_lst))
    avg_rmse = np.mean(rmse_list)
    rmse_std = np.std(rmse_list)
    
    print(f'RMSE:  {avg_rmse}')
    print(f'RMSE std:  {rmse_std}')
    print(f'interval: {[avg_rmse-rmse_std, avg_rmse+rmse_std]}')

    return avg_rmse,rmse_std

In [25]:
def total_corr(all_data_test_dict):
    corr_dict = {}
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        corr = stats.pearsonr(y_pred,y_actual)[0]
        corr_dict[key] =  corr
    
    total_corr = sum(corr_dict.values())
    #avg_corr = total_corr/len(list(all_data_test_dict.keys()))
    
    num_high_corr = 0
    for category in corr_dict:
        if corr_dict[category] >= 0.5:
            num_high_corr +=1
    
    print(f'Number of categories with High Correlation: {num_high_corr}')
    
    return total_corr

In [26]:
all_categories = list(all_data_test_dict.keys())

In [27]:
avg_rmse(all_data_test_dict)

RMSE:  2.0419314643664808
RMSE std:  1.7763568562394911
interval: [0.26557460812698963, 3.818288320605972]


(2.0419314643664808, 1.7763568562394911)

In [15]:
total_corr(all_data_test_dict)

Number of categories with High Correlation: 59


80.87969460731891

In [16]:
len(all_categories)

293

In [17]:
plot_results(all_data_test_dict, all_categories)

Category is: Electricity
RMSE is: 1.412400251872152


Category is: Infant formula
RMSE is: 1.1594780159970335


Category is: Recreational services
RMSE is: 0.95241689593492


Category is: Apples
RMSE is: 4.027057466780192


Category is: Other household appliances
RMSE is: 2.809858304884974


...

Category is: Toiletry items and cosmetics
RMSE is: 1.4036871377390263


Category is: Communications
RMSE is: 2.0999957554928246


Category is: All other miscellaneous food preparations
RMSE is: 1.5129314921806516


Category is: Cellular services
RMSE is: 4.077714104271412


Category is: All-items
RMSE is: 0.4205843017294529


In [18]:
def avg_r_squared(all_data_test_dict):
    r_squared_lst = []
    low_r_squared_indent = []
    positive_r_squared_indent = []
    high_r_squared_indent = []
    for key in all_categories:
        #print(f'category: {key}')
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        r2 = r2_score(y_actual, y_pred)
        r_squared_lst.append(r2)

        if r2<=0:
            low_r_squared_indent.append(df_predictions[df_predictions['Year']==2023]['Indent'].unique().item())
        
        else:
            positive_r_squared_indent.append(df_predictions[df_predictions['Year']==2023]['Indent'].unique().item())


        if r2>=0.5:
            high_r_squared_indent.append(df_predictions[df_predictions['Year']==2023]['Indent'].unique().item())

        if key =='All-items':
            headline_r2 = r2
    
    avg_r2_score = np.mean(r_squared_lst)
    r2_std = np.std(r_squared_lst)
    
    print(f'Average R Squared:  {avg_r2_score}')
    print(f'R Squared std:  {r2_std}')
    print(f'interval: {[avg_r2_score-r2_std, avg_r2_score+r2_std]}')
    print('--------------------------------------------------------')
    #print(f'sorted r2 list: {sorted(r_squared_lst, reverse=True)}')
    print(f'Headline R2: {headline_r2}')
    print(f'R2 list percentiles:\n[10: {np.percentile(r_squared_lst, 10)}, 25: {np.percentile(r_squared_lst, 25)}, 50: {np.percentile(r_squared_lst, 50)}, 75: {np.percentile(r_squared_lst, 75)}, 90: {np.percentile(r_squared_lst, 90)}]')
    print(f'num of neg/zero R^2: {sum(1 for i in r_squared_lst if i<= 0)}')
    print(f'num of postive R^2: {sum(1 for i in r_squared_lst if i> 0)}')
    print(f'num of high R^2: {sum(1 for i in r_squared_lst if i>= 0.5)}')


    #return headline_r2, low_r_squared_indent, positive_r_squared_indent, high_r_squared_indent

In [19]:
avg_r_squared(all_data_test_dict)

Average R Squared:  -0.39084502171190394
R Squared std:  2.614628225163456
interval: [-3.00547324687536, 2.223783203451552]
--------------------------------------------------------
Headline R2: 0.14837253928271488
R2 list percentiles:
[10: -0.9192484098442056, 25: -0.37430592616601444, 50: -0.05954315969594548, 75: 0.08425447524021812, 90: 0.2585689454067667]
num of neg/zero R^2: 167
num of postive R^2: 126
num of high R^2: 7


In [20]:
df_predictions = all_data_test_dict['All-items']
y_pred = df_predictions['Prediction'].values
y_actual = df_predictions['Actual'].values
mse = mean_squared_error(y_pred, y_actual)
rmse = np.sqrt(mse)


print(f'rmse: {rmse}')

rmse: 0.4205843017294529


In [21]:
def create_rmse_table(all_data_test_dict):
    categories = []
    indents = []
    mses = []
    rmses = []
    for key in all_categories:
        categories.append(key)

        df_predictions = all_data_test_dict[key]
        indents.append(df_predictions.Indent.unique().item())
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)

        mses.append(mse)
        rmses.append(np.sqrt(mse))
    
    series1 = pd.DataFrame(categories).reset_index(drop=True).rename(columns={0:'Category'})
    series2 = pd.DataFrame(indents).reset_index(drop=True).rename(columns={0:'Indent'})
    series3 = pd.DataFrame(mses).reset_index(drop=True).rename(columns={0:'MSE'})
    series4 = pd.DataFrame(rmses).reset_index(drop=True).rename(columns={0:'RMSE'})

    df1 = pd.concat([series1, series2], axis = 1)
    df2 = pd.concat([df1, series3], axis = 1)
    df3 = pd.concat([df2, series4], axis = 1)

    return df3


In [22]:
from pathlib import Path
def rmdir(dirc):
    dirc = Path(dirc)
    for itm in dirc.iterdir():
        if itm.is_dir():
            rmdir(itm)
            print("Deleting", itm, ".... successful.")
        else:
            itm.unlink()
    dirc.rmdir()

In [23]:
path = os.getcwd()

desired_path = path + '/rmse_per_category_hgru_canada.csv'

rmse_per_cat = create_rmse_table(all_data_test_dict)
rmse_per_cat.to_csv(desired_path, index=False)